In [6]:
from pathlib import Path
from tqdm import tqdm
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
import numpy  as np
import torch
import math
import warnings
from multiprocessing import Pool
warnings.filterwarnings('ignore')

In [7]:
images_dir = Path("/mnt/U/Datasets/lrs3pretrain/test_images/")
outpath = images_dir.parent / "test_cropped_images"

In [8]:
speakers = list(images_dir.glob("*"))
im_per_spk = {x.name: list(x.glob("*.jpg")) for x in speakers}
im_paths = []
for speaker, ims in im_per_spk.items():
    for im in ims:
        out = outpath / speaker / im.name
        if not out.exists():
            im_paths += [(im, str(out))]

In [9]:
model = InceptionResnetV1(pretrained='vggface2', device="cuda").eval()
mtcnn = MTCNN(margin=10, post_process=False, device="cuda")

In [10]:
batch_size = 512
p = Pool(16)
skipped = im_paths.pop(0)
pbar = tqdm(total=len(im_paths))
for i in range(math.ceil(len(im_paths)/batch_size)):
    ims, paths = list(zip(*im_paths[i*batch_size:(i+1)*batch_size]))
    images = list(p.map(Image.open, ims))
    mtcnn(images, save_path=paths)
    pbar.update(batch_size)



26624it [02:50, 227.12it/s]00<?, ?it/s]
 79%|███████▉  | 512/650 [00:03<00:00, 165.42it/s]
1024it [00:03, 214.34it/s]                        
1024it [00:20, 214.34it/s]